https://oauth.vk.com/authorize?client_id=6052784&scope=271502&redirect_uri=https://oauth.vk.com/blank.html&display=page&v=5.63&response_type=token

In [ ]:
# copy token here
token = "b26aa5422a903025651eed18bab686470c0e931156befe595e9d6c3728ae9bd67a9a4d4bfc05376982930"

In [ ]:
poll_group_id = "-172053584"

# VK API

In [ ]:
import vk
from vk.exceptions import VkAPIError

In [ ]:
session = vk.Session(token)
api = vk.API(session, v="5.74")

# Getting the whole wall

### Creating data dir if it doesn't exist

In [ ]:
import os

In [ ]:
os.makedirs("data", exist_ok=True)

### Requesting first 100

In [ ]:
kwargs = {"owner_id": poll_group_id, "count": "100"}

In [ ]:
group_wall_json = api.wall.get(**kwargs)

In [ ]:
len(group_wall_json["items"])

In [ ]:
# group_wall_json

In [ ]:
import time

while len(group_wall_json["items"]) < group_wall_json["count"]:
    kwargs.update({
        "offset": len(group_wall_json["items"])
    })
    extra_group_wall_json = api.wall.get(**kwargs)
    group_wall_json["items"].extend(extra_group_wall_json["items"])
    time.sleep(3)

Saving the result:

In [ ]:
import json

In [ ]:
with open("data/group_wall.json", "w") as f:
    json.dump(group_wall_json, f)

# Getting poll voters

In [ ]:
with open("data/group_wall.json", "r") as f:
    group_wall_json = json.load(f)

In [ ]:
# wall_post = group_wall_json["items"][1]
# poll = get_poll(wall_post)
# poll_voters = api.polls.getVoters(**kwargs)
# poll_voters

In [ ]:
def get_poll(wall_post):
    if "attachments" not in wall_post:
        return None
    for attachment in wall_post["attachments"]:
        if attachment["type"] != "poll":
            continue
        poll = attachment["poll"]
        return poll
    return None

In [ ]:
for i, wall_post in enumerate(group_wall_json["items"]):
    poll = get_poll(wall_post)
    if poll is None or poll["anonymous"]:
        continue
    kwargs = {
        "poll_id": poll["id"],
        "owner_id": poll["owner_id"],
        "count": 1000,
        "answer_ids": list(map(lambda answer: answer["id"], poll["answers"])),
        "fields": "nickname, screen_name, sex, photo, photo_medium, photo_big"
    }
    
    try:
        poll_voters = api.polls.getVoters(**kwargs)
    except VkAPIError as vk_error:
#         print(vk_error)
        time.sleep(1)
        continue
    with open("data/poll_{}.json".format(poll["id"]), "w") as f:
        json.dump(poll_voters, f)
    if (i + 1) % 20 == 0:
        print("Processed {}/{} polls".format(i + 1, len(group_wall_json["items"])))
    time.sleep(5)

# Processing and squeezing data

### Squeezing all users:

In [ ]:
all_ids_to_users = {}
all_ids = []
for filename in os.listdir("data"):
    if "poll_" not in filename:
        continue
    poll_id = int(filename.split("poll_")[1].split(".")[0])
    with open("data/" + filename, "r") as f:
        poll = json.load(f)
    
    if not isinstance(poll, list):
        print("error")
    
    answers_to_ids = []
    for answer in poll:
        answer_id_to_user = list(map(lambda vote: (vote["id"], vote), answer["users"]["items"]))
        all_ids_to_users.update(dict(answer_id_to_user))
        all_ids.extend(map(lambda item: item[0], answer_id_to_user))

In [ ]:
from collections import Counter
cnt = Counter(all_ids)

In [ ]:
for user_id in cnt:
    all_ids_to_users[user_id]["num_votes"] = cnt[user_id]

In [ ]:
with open("data/users.json", "w") as f:
    json.dump(all_ids_to_user, f)

### Building "sparse" relativity matrix.

In [ ]:
polls = []
for filename in os.listdir("data"):
    if "poll_" not in filename:
        continue
    poll_id = int(filename.split("poll_")[1].split(".")[0])
    with open("data/" + filename, "r") as f:
        poll = json.load(f)
    
    if not isinstance(poll, list):
        print("error")
    
    answers_to_ids = []
    for answer in poll:
        answers_to_ids.append(list(map(lambda vote: vote["id"], answer["users"]["items"])))
    polls.append(answers_to_ids)

Now we have variable 'polls' which is [POLL_ID \* ANSWER_ID \* USER] where ids are not real but just indexation.

In [ ]:
with open("data/vote_matrix.json", "w") as f:
    json.dump(polls, f)